In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import train_test_split
import warnings
import plotly.express as px
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#data = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/o2Saturation.csv')
#data


In [ ]:
data = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
data

In [ ]:
data.var()

In [ ]:
data.describe(include='all')

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
correlation= data.corr()
correlation

In [ ]:
fig = plt.figure(figsize = (15,8))
sns.heatmap(correlation, annot = True)

In [ ]:

fig = plt.figure(figsize = (15,8))
data.boxplot()

In [ ]:
df1 = data.select_dtypes([np.int, np.float])
for i, col in enumerate(df1.columns):
    plt.figure(i)
    sns.distplot(df1[col])

In [ ]:
# # Log transformation
# data["log_age"]= np.log(data['age'])
# data["log_trtbps"]= np.log(data['trtbps'])
# data["log_chol"]= np.log(data['chol'])
# data["log_thalachh"]= np.log(data['thalachh'])


In [ ]:
# data1 = data.copy()

In [ ]:
# data = data.drop(['age', 'trtbps', 'chol', 'thalachh'], axis = 1)

In [ ]:
for i in data.columns:
    print(f'{i}{data[i].unique()}')

In [ ]:
X = data.drop(['output'], axis =1)
y = data['output']
X

In [ ]:
x_scaled = StandardScaler()
x_scaled.fit(X)
x_scale = x_scaled.transform(X)
x_scale
Xdata_scaled = pd.DataFrame(x_scale, columns=X.columns)
Xdata_scaled

In [ ]:
plt.figure(figsize=(12,8))
Xdata_scaled.boxplot()

In [ ]:
def test_models(models, X,y, iterations = 100):
    results = {}
    for i in models:
        r2_train = []
        r2_test = []
        for j in range(iterations):
            x_train, x_test, y_train, y_test = train_test_split(X,y,test_size= 0.2)
            models[i].fit(x_train,y_train).predict(x_train)
            r2_train.append(models[i].fit(x_train,y_train).score(x_train, y_train))
            models[i].fit(x_train,y_train).predict(x_test)
            r2_test.append(models[i].fit(x_train,y_train).score(x_test, y_test))
            
            results[i] = [np.mean(r2_train), np.mean(r2_test)]
           
    return pd.DataFrame(results)

In [ ]:
models = {'LG': LogisticRegression(),'Ridgeclassifier': RidgeClassifier()}

test_models(models, Xdata_scaled, y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(Xdata_scaled,y,test_size= 0.25, random_state=5)

## Hyperparameter turning for LogisticRegression

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2','10', '15']
c_values = [100, 10, 1.0, 0.1, 0.01]
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(x_train, y_train)
print(grid_result.best_params_)
print(grid_result.score(x_test, y_test))

## Hyperparameter turning for Ridgeclassifier

In [ ]:
model = RidgeClassifier()
alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# define grid search
grid = dict(alpha=alpha)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(x_train, y_train)

print(grid_result.best_params_)
print(grid_result.score(x_test, y_test))

## Hyperparameter turning for GradientBoostingClassifier

In [ ]:
parameters = {
    "n_estimators":[5,50,250,500],
    "max_depth":[1,3,5,7,9],
    "learning_rate":[0.01,0.1,1,10,100]}

gb = GradientBoostingClassifier()

gb_search = GridSearchCV(estimator = gb, param_grid = parameters, 

                          cv = 10, n_jobs = -1, return_train_score=True)

gb_search.fit(x_train, y_train)

print(gb_search.best_params_)
print(gb_search.score(x_test, y_test))

## hyperparameter turning for Decision Tree Classification

In [ ]:
param_grid = {"max_depth": [3, None],
              "max_features": randint(1, 9),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}

tree = DecisionTreeClassifier()

# Instantiate the RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(tree, param_grid, cv=5)

# Fit it to the data
tree_cv.fit(x_train, y_train)

print(tree_cv.best_params_)
print(tree_cv.score(x_test, y_test))

##  Hyperparameter tuning for RandomForestClassification

In [ ]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {  'max_depth': np.arange(10, 100, 10),
              'max_features': ['auto',' log2'], 'n_estimators': [25,30, 35, 50, 100]}

rfc = RandomForestClassifier()

g_search = GridSearchCV(estimator = rfc, param_grid = param_grid, 

                          cv = 5, n_jobs = -1, return_train_score=True)

g_search.fit(x_train, y_train)

print(g_search.best_params_)
print(g_search.score(x_test, y_test))

##  Hyperparameter tuning for SVM

In [ ]:
model = SVC()
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10, 1.0, 0.1, 0.01]
gamma = ['scale']
# define grid search
grid = dict(kernel=kernel,C=C,gamma=gamma)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(x_train, y_train)
print(grid_result.best_params_)
print(grid_result.score(x_test, y_test))

##  Hyperparameter tuning for KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
#List Hyperparameters that we want to tune.
#leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
#p=[1,2]
#Convert to dictionary
hyperparameters = dict(n_neighbors=n_neighbors)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=3)
#Fit the model
grid_result = clf.fit(x_train,y_train)

print(grid_result.best_params_)
print(grid_result.score(x_test, y_test))

## Logistic Regression

In [ ]:
lgc = LogisticRegression()
lgc.fit(x_train, y_train)
y_test_pred = lgc.predict(x_test)
accuracy_score(y_test,y_test_pred)

## Gradient boosting classifier

In [ ]:
gbc = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 1, n_estimators = 500)

In [ ]:
gbc.fit(x_train, y_train)
y_test_pred = gbc.predict(x_test)
accuracy_score(y_test, y_test_pred)


## **During the hyperparameter turning I found a best parameters to get best scores for several models. Out of all models I found best score for LogisticRegression, Ridgeclassifier, GradientBoostingClassifier and Randomforestclassifier have acuuracy score of 0.91.**